# Background subtraction

This notebook implements the method developed in script Background.ipynb in a loop to handle a batch of images.

It expects .ARW images as input, and generates corresponding .bkg_subtracted.fits files.

In [1]:
import os, glob
import time, datetime
import math

import multiprocessing as mp
from multiprocessing import Pool

import numpy as np

from scipy import stats

from astropy.io import fits
from astropy.table import Table
from astropy.stats import SigmaClip
from astropy.convolution import Gaussian2DKernel, interpolate_replace_nans

from photutils.background import Background2D, MedianBackground, ModeEstimatorBackground

import rawpy
import exifread

from worker_background import Worker
from datapath import DATA

## Initialization

In [2]:
# these were determined in script White_light_images as the normalization
# factors that render the smoothest background.
red_norm =  1.34
blue_norm = 1.25

# lets try with best for star @ 4300K
# red_norm =  1.9
# blue_norm = 1.9

# parameters to control background subtraction
bkg_cell_footprint = (35, 35)
bkg_filter = (9, 9)

bkg_sigma_clip = SigmaClip(sigma=5.)
bkg_kernel = Gaussian2DKernel(x_stddev=5)
bkg_estimator = ModeEstimatorBackground()

In [3]:
# paths
data_dirpath = os.path.join(DATA,'astrophotography_data/Andromeda_2022/104MSDCF')

input_suffix = '.ARW'
output_suffix = '.bkg_subtracted.fits'

In [4]:
# images to subtract background from
image_list = list(glob.glob(data_dirpath + '/*' + input_suffix))
image_list.sort()

image_list

['/Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09939.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09940.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09941.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09942.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09943.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09944.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09945.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09946.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09947.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09948.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09949.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/104M

In [5]:
# read base image - we need this in order to access the camera color matrix
raw = rawpy.imread(image_list[int(len(image_list)/2)]) # mid-point
imarray_base = raw.raw_image_visible.astype(float)

In [6]:
# masks that isolate the RGB pixels
colors_array = raw.raw_colors_visible

red_mask = np.where(colors_array == 0, 1, 0)

green_mask_1 = np.where(colors_array == 1, 1, 0)
green_mask_2 = np.where(colors_array == 3, 1, 0)
green_mask = green_mask_1 | green_mask_2

blue_mask = np.where(colors_array == 2, 1, 0)

## Functions

All functions are defined in module 'worker_background.py'. The multiprocessing framework requires that everything that runs on a separate process has to have its own address space. 

## Main loop

In [7]:
# for image_name in image_list:
#     raw = rawpy.imread(image_name)
#     imarray = raw.raw_image_visible.astype(float)
#     subtracted = subtract_background(imarray, red_norm=red_norm, blue_norm=blue_norm)
#     write_bkg_subtracted(image_name, subtracted)

# number of processors
nproc = 8    # (Mac M1)

# split file list in nproc chuncks; one for each processor
image_list_chuncks = np.array_split(image_list, nproc)

results = []
pool = Pool(nproc)

for p in range(nproc):
    worker = Worker(image_list_chuncks[p], input_suffix, output_suffix, 
                    red_mask, green_mask, blue_mask, 
                    red_norm, blue_norm,
                    bkg_cell_footprint, bkg_filter, bkg_sigma_clip, bkg_kernel, bkg_estimator)

    r = pool.apply_async(worker)
    results.append(r)

for r in results:
    r.wait()

pool.close()    

Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09939.bkg_subtracted.fits Mon Sep 11 13:20:28 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09955.bkg_subtracted.fits Mon Sep 11 13:20:29 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09963.bkg_subtracted.fits Mon Sep 11 13:20:30 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09971.bkg_subtracted.fits Mon Sep 11 13:20:31 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09947.bkg_subtracted.fits Mon Sep 11 13:20:31 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09979.bkg_subtracted.fits Mon Sep 11 13:20:31 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Androme

Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09985.bkg_subtracted.fits Mon Sep 11 13:36:10 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09953.bkg_subtracted.fits Mon Sep 11 13:36:12 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09946.bkg_subtracted.fits Mon Sep 11 13:38:19 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09962.bkg_subtracted.fits Mon Sep 11 13:38:25 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09978.bkg_subtracted.fits Mon Sep 11 13:38:35 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/104MSDCF/DSC09970.bkg_subtracted.fits Mon Sep 11 13:38:35 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Androme